In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'

# Getting the papers

In [2]:
import os
import json
from tqdm.auto import tqdm

In [3]:
# get this folder from the arabic arabs dataset by cloning the repo and replace its path here
articles_dir = '../arabs-dataset/articles'

In [4]:
papers_paths_json = []
papers_metdatas = []
for journal in tqdm(os.listdir(articles_dir)):
    for volume in os.listdir(f"{articles_dir}/{journal}"):
        volume_date = volume.split('-')[-1]
        if 2010 <= int(volume_date) <= 2022: # consider this range
            for issue in os.listdir(f"{articles_dir}/{journal}/{volume}"):
                for paper_path in os.listdir(f"{articles_dir}/{journal}/{volume}/{issue}"):
                    if 'json' not in paper_path: # in case it is not json (i.e. a .pdf file)
                        continue
                    paper_path_json = f"{articles_dir}/{journal}/{volume}/{issue}/{paper_path}"
                    try:
                        paper_metadata = json.load(open(paper_path_json))
                    except Exception as e:
                        continue
                    papers_metdatas.append(paper_metadata)
                    papers_paths_json.append(paper_path_json)

  0%|          | 0/354 [00:00<?, ?it/s]

In [5]:
len(papers_metdatas), len(papers_paths_json), len([metadata for metadata in papers_metdatas if metadata.get('paper_extracted_content')])

(129971, 129971, 14400)

In [6]:
filtered_papers_metadata = [metadata for metadata in papers_metdatas if metadata.get('paper_extracted_content') and  metadata.get('arabic_abstract')]
filtered_papers_metadata = [
    metadata for metadata in filtered_papers_metadata 
    if not metadata.get('arabic_abstract') or  # Keep papers without Arabic abstract
    (75 <= len(metadata['arabic_abstract'].split()) <= 300)  # Keep papers with valid Arabic abstract length
]
len(filtered_papers_metadata)

9993

In [7]:
len(set([metadata['title'] for metadata in filtered_papers_metadata]))

9974

In [8]:
unique_filtered_papers_metdata = []
for metadata in tqdm(filtered_papers_metadata):
    if metadata['title'] not in [metadata['title'] for metadata in unique_filtered_papers_metdata]:
        unique_filtered_papers_metdata.append(metadata)
len(unique_filtered_papers_metdata)

  0%|          | 0/9993 [00:00<?, ?it/s]

9974

In [9]:
filtered_papers_metadata = unique_filtered_papers_metdata

# Filter dataset with Jais for readable Arabic paper content

In [10]:
import sys
sys.path.append('/home/majed_alshaibani/Projects/jrcai_corekit/src')

In [11]:
from llm import *

/home/majed_alshaibani/Projects/ai-content-detection-dataset/venv/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [ ]:
llm_loader = LLMLoader(
    '/hdd/shared_models/Meta-Llama-3.1-8B-Instruct/',
    llm_initializer=Llama31Initializer(),
    )
messsage_generator = MessageGeneratorFromLocalLLM(llm_loader)

In [13]:
def get_chat_completion(messages):
    llm_response = messsage_generator(messages)
    return llm_response[0]

In [14]:
print(get_chat_completion(
    messages=[[{
            'role': 'user',
            'content': 'ما هو لون نجوم السماء؟'
        }]]
    )
)

loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/config.json
Model config LlamaConfig {
  "_name_or_path": "/hdd/shared_models/Meta-Llama-3.1-70B-Instruct/",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden_layers": 80,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.43.3",
  "use_cac

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file /hdd/shared_models/Meta-Llama-3.1-70B-Instruct/generation_config.json
Generate 

Meta-Llama-3.1-70B-Instruct - batches sizes = [1]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 1/1 [00:02<00:00,  2.34s/it]

لون النجوم مختلف، ولكن معظمها أبيض.


In [15]:
def validate_paper_content(validation_prompts, batch_size=10):   
    # Prepare batches of prompts starting from the last completed batch
    batched_messages = [
        [
            [{'role': 'user', 'content': prompt}] 
            for prompt in validation_prompts[i:i+batch_size]
        ]
        for i in range(0, len(validation_prompts), batch_size)
    ]
    
    # Main process
    results = []
    for batch in tqdm(batched_messages, desc="Processing batches"):
        generated_batch = messsage_generator(batch)
        batch_validated_contents = []
        
        for answer in generated_batch:
            batch_validated_contents.append(answer.strip())
        
        # Extend the accumulated results and save after each batch
        results.extend(batch_validated_contents)
    return results

In [ ]:
prompt = """
أنت خبير في تحليل النصوص العربية المستخرجة من ملفات PDF. مهمتك هي تقييم مدى وضوح النص المقدم وقابليته للفهم.

معلومات مهمة:
النص المقدم مستخرج آلياً من ملف PDF وقد يحتوي على أخطاء شائعة مثل:
- تداخل الحروف العربية (مثال: "في الحالات الأخرى" تصبح "في الحاالت األخرى")
- تكرار الحروف بشكل عشوائي (مثال: السسسسلام علييييكم)
- تحويل خاطئ للحروف المتشابهة (مثال: "ه" تصبح "ة")
- فقدان النقاط على الحروف (مثال: "ي" تصبح "ى")
- ظهور رموز غريبة (مثال: ال%حمد لل@ه)
- دمج أو تقطيع خاطئ للكلمات (مثال: فيمابينهم، ال قرآن)

المطلوب:
اقرأ النص المقدم وأجب بكلمة واحدة فقط دون أي إضافات:
- اكتب "نعم" (بدون أقواس أو نقطة أو شرح) إذا كان النص مفهوماً
- اكتب "لا" (بدون أقواس أو نقطة أو شرح) إذا كان النص غير مفهوم

ملاحظات هامة:
- ممنوع إضافة أي شرح أو تعليق
- ممنوع استخدام الأقواس ()
- ممنوع إضافة نقطة في النهاية
- ممنوع إضافة أسطر فارغة
- يجب أن تكون الإجابة كلمة واحدة فقط: إما "نعم" أو "لا"

النص المقدم:
{content}

الإجابة:
""".strip()

In [74]:
abstracts = unique_filtered_papers_metdata
len(abstracts)

9974

In [75]:
print(prompt.format(content=' '.join(abstracts[0]['paper_extracted_content'].split(' ')[:1000])))

أنت خبير في تحليل النصوص العربية المستخرجة من ملفات PDF. مهمتك هي تقييم مدى وضوح النص المقدم وقابليته للفهم.

المطلوب:
اقرأ النص العربي المقدم وقيّم درجة وضوحه وقابليته للفهم على مقياس من 0 إلى 100:
- 100 يعني نص عربي سليم تماماً ومفهوم بشكل كامل
- 0 يعني نص غير مفهوم إطلاقاً
- القيم بينهما تعبر عن درجات متفاوتة من الوضوح

ملاحظات هامة:
- يجب أن تكون الإجابة رقماً فقط من 0 إلى 100
- لا تضف أي كلمات أو علامات أو شروحات مع الرقم

النص المقدم:
مجلة المعارف للبحوث والدراسات التاريخية مجلة دورية دولية محكمة العدد 6< =6 صور عننظام التعليم عند المرأة األندلسية أ- يـمانـي رشـيد ، قسم التاريخ ، جامعة تلمسان . تمهـيد : كثَتا ما ارتبطت ا١تصادر التارخيية يف األندلس خاصة منها كتب الًتاجم والفهرسات والربامج وغَتىا بدراسة حياة العلماء و الرواة والقضاة و الساسة ؛ وقد تطورت ىذه ا١تادة حىت ترك لنا ا١تؤلفون األندلسيون سلسلة متواصلة اٟتلقات من كتب التـراجم كالصلة البن بشكوال ، وصلة الصلة البن الزبَت، والتكملة لكتاب الصلة البن اآلبار، والذيل والتكملة لكتايب ا١توصول و الصلة البن عبد ا١تلك ا١تراكشي إضافة إىل

In [76]:
validation_prompts = [
    prompt.format(content=' '.join(abstract['paper_extracted_content'].split(' ')[:1000]))
    for abstract in abstracts
]
len(validation_prompts)

9974

In [77]:
validation_prompts[:3]

['أنت خبير في تحليل النصوص العربية المستخرجة من ملفات PDF. مهمتك هي تقييم مدى وضوح النص المقدم وقابليته للفهم.\n\nالمطلوب:\nاقرأ النص العربي المقدم وقيّم درجة وضوحه وقابليته للفهم على مقياس من 0 إلى 100:\n- 100 يعني نص عربي سليم تماماً ومفهوم بشكل كامل\n- 0 يعني نص غير مفهوم إطلاقاً\n- القيم بينهما تعبر عن درجات متفاوتة من الوضوح\n\nملاحظات هامة:\n- يجب أن تكون الإجابة رقماً فقط من 0 إلى 100\n- لا تضف أي كلمات أو علامات أو شروحات مع الرقم\n\nالنص المقدم:\nمجلة المعارف للبحوث والدراسات التاريخية مجلة دورية دولية محكمة العدد 6< =6 صور عننظام التعليم عند المرأة األندلسية أ- يـمانـي رشـيد ، قسم التاريخ ، جامعة تلمسان . تمهـيد : كثَتا ما ارتبطت ا١تصادر التارخيية يف األندلس خاصة منها كتب الًتاجم والفهرسات والربامج وغَتىا بدراسة حياة العلماء و الرواة والقضاة و الساسة ؛ وقد تطورت ىذه ا١تادة حىت ترك لنا ا١تؤلفون األندلسيون سلسلة متواصلة اٟتلقات من كتب التـراجم كالصلة البن بشكوال ، وصلة الصلة البن الزبَت، والتكملة لكتاب الصلة البن اآلبار، والذيل والتكملة لكتايب ا١توصول و الصلة البن عبد ا١تلك ا١ت

In [81]:
understood_metadata_results = validate_paper_content(validation_prompts[:25],batch_size=10)

Processing batches:   0%|          | 0/3 [00:00<?, ?it/s]

Meta-Llama-3.1-70B-Instruct - batches sizes = [3, 3, 3, 1]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 4/4 [00:20<00:00,  5.16s/it]


Meta-Llama-3.1-70B-Instruct - batches sizes = [2, 3, 3, 2]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 4/4 [00:25<00:00,  6.42s/it]


Meta-Llama-3.1-70B-Instruct - batches sizes = [2, 3]


TextGeneration using Meta-Llama-3.1-70B-Instruct: 100%|██████████| 2/2 [00:12<00:00,  6.08s/it]


In [84]:
for i, p in enumerate(validation_prompts[:25]):
    print(i, validation_prompts[i])

0 أنت خبير في تحليل النصوص العربية المستخرجة من ملفات PDF. مهمتك هي تقييم مدى وضوح النص المقدم وقابليته للفهم.

المطلوب:
اقرأ النص العربي المقدم وقيّم درجة وضوحه وقابليته للفهم على مقياس من 0 إلى 100:
- 100 يعني نص عربي سليم تماماً ومفهوم بشكل كامل
- 0 يعني نص غير مفهوم إطلاقاً
- القيم بينهما تعبر عن درجات متفاوتة من الوضوح

ملاحظات هامة:
- يجب أن تكون الإجابة رقماً فقط من 0 إلى 100
- لا تضف أي كلمات أو علامات أو شروحات مع الرقم

النص المقدم:
مجلة المعارف للبحوث والدراسات التاريخية مجلة دورية دولية محكمة العدد 6< =6 صور عننظام التعليم عند المرأة األندلسية أ- يـمانـي رشـيد ، قسم التاريخ ، جامعة تلمسان . تمهـيد : كثَتا ما ارتبطت ا١تصادر التارخيية يف األندلس خاصة منها كتب الًتاجم والفهرسات والربامج وغَتىا بدراسة حياة العلماء و الرواة والقضاة و الساسة ؛ وقد تطورت ىذه ا١تادة حىت ترك لنا ا١تؤلفون األندلسيون سلسلة متواصلة اٟتلقات من كتب التـراجم كالصلة البن بشكوال ، وصلة الصلة البن الزبَت، والتكملة لكتاب الصلة البن اآلبار، والذيل والتكملة لكتايب ا١توصول و الصلة البن عبد ا١تلك ا١تراكشي إضافة إ

In [23]:
len([metadata for metadata, result in zip(filtered_papers_metadata, understood_metadata_results) if result.strip() == 'نعم'])

5287

Note that this number (5287) might change slightly. This may happen on the samples that the LLM is not 100 sure about.

In [24]:
unique_understood_filtered_papers_metadata = []
for metadata, result in zip(filtered_papers_metadata, understood_metadata_results):
    if result == 'نعم':
        unique_understood_filtered_papers_metadata.append(metadata)
len(unique_understood_filtered_papers_metadata)

5287

# Final Selection

randomize

In [25]:
GLOBAL_SEED = 42

In [26]:
import random
random.seed(GLOBAL_SEED)

In [27]:
random.shuffle(unique_understood_filtered_papers_metadata)

In [28]:
filtered_papers_metadata = filtered_papers_metadata[:3000] # take only the first 3k

In [29]:
len(set([metadata['title'] for metadata in filtered_papers_metadata]))

3000

# Descriptive Statistics

In [30]:
english_only_counter = 0
arabic_only_counter = 0 
english_arabic_counter = 0

# Lists to store lengths for statistics
arabic_abstract_lengths = []

for metadata in filtered_papers_metadata:
    if metadata.get('english_abstract') and not metadata.get('arabic_abstract'):
        english_only_counter += 1
    elif metadata.get('arabic_abstract') and not metadata.get('english_abstract'):
        arabic_only_counter += 1
        # Add length of Arabic abstract to list
        arabic_abstract_lengths.append(len(metadata['arabic_abstract'].split(' ')))
    elif metadata.get('english_abstract') and metadata.get('arabic_abstract'):
        english_arabic_counter += 1
        # Add length of Arabic abstract to list
        arabic_abstract_lengths.append(len(metadata['arabic_abstract'].split(' ')))
    else:
        print(metadata['arabic_abstract'], metadata['english_abstract'])

# Calculate statistics if we have any Arabic abstracts
min_arabic_length = min(arabic_abstract_lengths)
max_arabic_length = max(arabic_abstract_lengths)
avg_arabic_length = sum(arabic_abstract_lengths) / len(arabic_abstract_lengths)

# Return all counters and statistics
english_only_counter, arabic_only_counter, english_arabic_counter, min_arabic_length, max_arabic_length, avg_arabic_length

(0, 1619, 1381, 75, 294, 119.126)

In [31]:
# save these filtered metadata
!mkdir -p arabic_datasets
with open('arabic_datasets/arabic_filtered_papers.json', 'w') as f:
    json.dump(filtered_papers_metadata, f, ensure_ascii=False,indent=4)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
